In [1]:
import hashlib
import numpy as np

## Helper Function

In [2]:
def vector_to_circulant(v, F):
    """
    Creates a circulant matrix from a vector v over Field F.
    Row 0: [v0, v1, v2, ... vr-1]
    Row 1: [vr-1, v0, v1, ... vr-2] (Right Shift)
    ...
    """
    r = len(v)
    rows = []
    # Convert input vector to a list for easy shifting
    current_row = list(v)
    
    for i in range(r):
        rows.append(current_row)
        # Rotate right for the next row
        # [a, b, c] -> [c, a, b]
        current_row = [current_row[-1]] + current_row[:-1]
        
    return matrix(F, r, r, rows)

print("[vec-to-circ] function defined.")

[vec-to-circ] function defined.


## Key Generation

In [3]:
def bike_keygen(r, w):
    """
    Generates BIKE keys.
    r: Block size (Prime number, e.g., 53)
    w: Weight of secret keys (must be even, split between h0 and h1)
    """
    F = GF(2)
    
    # 1. Generate Sparse Secret Keys (h0, h1)
    # Each has weight w/2
    w_half = w // 2
    
    # Helper to make random sparse vector
    def make_sparse(length, weight):
        vec = vector(F, length * [0])
        import random
        idxs = random.sample(range(length), weight)
        for i in idxs: vec[i] = 1
        return vec

    # We loop until h0 is invertible (required for public key)
    while True:
        h0 = make_sparse(r, w_half)
        h1 = make_sparse(r, w_half)
        
        # Convert to Circulant Matrices
        H0 = vector_to_circulant(h0, F)
        H1 = vector_to_circulant(h1, F)
        
        if H0.is_invertible():
            break
            
    # 2. Compute Public Key: H_pub = H1 * H0_inv
    H_pub = H1 * H0.inverse()
    
    # Private Key contains the vectors and matrices needed for decoding
    priv_key = {
        'h0': h0, 'h1': h1, 
        'H0': H0, 'H1': H1,
        'r': r
    }
    
    return H_pub, priv_key

print("[KeyGen] function defined")

[KeyGen] function defined


## Encapsulation

In [4]:
def bike_encapsulate(H_pub, t):
    """
    Bob creates a shared secret and encrypts it.
    t: Error threshold (weight of e0 and e1 combined)
    """
    r = H_pub.nrows()
    F = H_pub.base_ring()
    
    # 1. Generate Error Vectors (e0, e1)
    # Usually split weight t roughly equally, or t total. 
    # For simplicity, we put floor(t/2) in e0 and ceil(t/2) in e1
    w_e0 = t // 2
    w_e1 = t - w_e0
    
    def make_error(length, weight):
        vec = vector(F, length * [0])
        import random
        idxs = random.sample(range(length), weight)
        for i in idxs: vec[i] = 1
        return vec
        
    e0 = make_error(r, w_e0)
    e1 = make_error(r, w_e1)
    
    # 2. Compute Ciphertext: c = e0 + e1 * H_pub
    # Note: Vector * Matrix multiplication in Sage is row_vector * matrix
    c = e0 + (e1 * H_pub)
    
    # 3. Derive Shared Secret
    # Hash (e0 || e1)
    secret_input = str(e0) + str(e1)
    shared_secret = hashlib.sha256(secret_input.encode('utf-8')).hexdigest()
    
    return c, shared_secret

print("[Encap] function defined")

[Encap] function defined


## Decapsulation

In [5]:
def bike_decapsulate(c, priv_key, t, max_iters=20):
    """
    Alice recovers e0, e1 using Bit Flipping.
    """
    H0 = priv_key['H0']
    H1 = priv_key['H1'] # Needed to compute syndrome part for e1 checks
    h0 = priv_key['h0']
    h1 = priv_key['h1']
    r = priv_key['r']
    F = GF(2)
    
    # 1. Compute Initial Syndrome
    # s = c * H0  (This works because c = e0 + e1*H1*H0_inv -> c*H0 = e0*H0 + e1*H1)
    syndrome = c * H0
    
    # Candidates for e0 and e1 (Start as all zeros)
    # We essentially decode 'in place' by tracking changes, 
    # but strictly speaking we are finding e0, e1 to subtract them.
    # In this simplified BFD, we will just try to find e0', e1' that produce the syndrome.
    e0_hat = vector(F, r*[0])
    e1_hat = vector(F, r*[0])
    
    # Pre-compute column indices where h0 and h1 are 1
    # This speeds up the "Counter" step significantly
    h0_locs = [i for i, val in enumerate(h0) if val == 1]
    h1_locs = [i for i, val in enumerate(h1) if val == 1]
    
    for iteration in range(max_iters):
        # Calculate current syndrome based on our guesses so far
        # current_S = s - (e0_hat*H0 + e1_hat*H1)
        # Note: In GF(2), subtraction is addition.
        current_S = syndrome + (e0_hat * H0) + (e1_hat * H1)
        
        # If syndrome is all zeros, we are done!
        if current_S.hamming_weight() == 0:
            break
            
        # --- ITERATIVE FLIPPING STEP ---
        # We need to count "Unsatisfied Parity Checks" (UPC) for every bit position
        
        # A bit i in e0 affects syndrome positions: (i + loc) % r for loc in h0_locs
        # We verify this by checking correlation
        
        upc_e0 = [0] * r
        upc_e1 = [0] * r
        
        # Count for e0
        for i in range(r):
            count = 0
            for loc in h0_locs:
                # The column logic in circulant matrices corresponds to (i + loc) % r 
                # (depending on transpose definition, simplified here as rotational matching)
                if current_S[(i + loc) % r] == 1:
                    count += 1
            upc_e0[i] = count
            
        # Count for e1
        for i in range(r):
            count = 0
            for loc in h1_locs:
                if current_S[(i + loc) % r] == 1:
                    count += 1
            upc_e1[i] = count
            
        # Determine Threshold (Simple Strategy: Max UPC)
        max_matches = max(max(upc_e0), max(upc_e1))
        
        # If max matches is too low, we might be stuck (or finished imperfectly)
        # For this toy version, we stop if threshold is very low to avoid infinite loops
        if max_matches <= len(h0_locs) // 2: 
            break
            
        # Flip bits that meet the threshold
        # We use a strict threshold (max_matches) to be safe
        threshold = max_matches
        
        for i in range(r):
            if upc_e0[i] >= threshold:
                e0_hat[i] += 1 # Flip (0->1 or 1->0)
        for i in range(r):
            if upc_e1[i] >= threshold:
                e1_hat[i] += 1 # Flip
                
    # 2. Final Output
    # The errors we found are e0_hat, e1_hat.
    # We define the recovered error as our guess.
    
    # Derive Shared Secret
    secret_input = str(e0_hat) + str(e1_hat)
    shared_secret = hashlib.sha256(secret_input.encode('utf-8')).hexdigest()
    
    return shared_secret

print("[Decap] function defined")

[Decap] function defined


In [6]:
# ==========================================
# 5. VERIFICATION TEST
# ==========================================
# Parameters for a "Toy" instance
r_toy = 53  # Block size (Prime)
w_toy = 6   # Weight of h0, h1 (Small & Even)
t_toy = 4   # Error weight

print(f"--- Running BIKE Test (r={r_toy}, w={w_toy}, t={t_toy}) ---")

# 1. KeyGen
H_pub, priv = bike_keygen(r_toy, w_toy)
print("Keys Generated.")

# 2. Encapsulate
c, secret_bob = bike_encapsulate(H_pub, t_toy)
print(f"Bob's Secret:   {secret_bob[:10]}...")

# 3. Decapsulate
secret_alice = bike_decapsulate(c, priv, t_toy)
print(f"Alice's Secret: {secret_alice[:10]}...")

# 4. Check
if secret_bob == secret_alice:
    print("SUCCESS: Secrets Match!")
else:
    print("FAILURE: Secrets do not match.")

--- Running BIKE Test (r=53, w=6, t=4) ---
Keys Generated.
Bob's Secret:   7c9afb14bd...
Alice's Secret: 7c9afb14bd...
SUCCESS: Secrets Match!


# Attack

In [7]:
import time
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sys

In [8]:
def bike_isd_check(H_pub, c, t, max_iters=1000):
    """
    Attempts to break BIKE using Prange's ISD algorithm.
    Returns True if successful, False otherwise.
    """
    r = H_pub.nrows()
    F = H_pub.base_ring()
    n_total = 2 * r
    
    # 1. Construct the Parity Check Matrix for Attack
    # H_attack = [H_pub^T | I]
    # We solve H_attack * e_col = c_col
    H_attack = block_matrix(F, 1, [H_pub.transpose(), identity_matrix(F, r)])
    
    # The target syndrome is the ciphertext (transposed)
    s = c
    
    for _ in range(max_iters):
        # 2. Pick a random Information Set (r columns indices)
        # Instead of shuffling the whole matrix (slow), we pick random columns
        import random
        idxs = random.sample(range(n_total), r)
        
        # 3. Extract submatrix
        H_sub = H_attack.matrix_from_columns(idxs)
        
        # 4. Check Invertibility
        if not H_sub.is_invertible():
            continue
            
        # 5. Solve: H_sub * e_sub = s  =>  e_sub = H_sub^-1 * s
        # Note: Sage handles 'Matrix * Vector' as Matrix * ColumnVector
        try:
            e_candidate = H_sub.inverse() * s
            
            # 6. Check Weight
            # Prange assumes ALL errors are located in the chosen columns.
            # So if the weight of this solution is <= t, we likely found it.
            if e_candidate.hamming_weight() == t:
                return True
        except:
            continue
            
    return False

print("[Attack] Function Defined.")

[Attack] Function Defined.


In [9]:
def run_bike_3param_experiment(r_list, t_list, w_list, trials=5, max_iters=1000):
    results_list = []
    
    # Sanity Check inputs
    if not r_list or not t_list or not w_list:
        print("ERROR: One of the input lists (r, t, or w) is empty!")
        return pd.DataFrame()

    total_configs = len(r_list) * len(t_list) * len(w_list)
    config_idx = 0
    
    print(f"--- Starting BIKE Testbed (Robust Mode) ---")
    print(f"Total Configurations: {total_configs}")
    print(f"r values: {r_list}")
    print(f"w values: {w_list}")
    print(f"t values: {t_list}")
    print("=" * 65)
    
    for r in r_list:
        r = int(r)
        
        for w in w_list:
            w = int(w)
            
            # CHECK 1: Is weight valid?
            if w % 2 != 0:
                print(f"[Skipping] r={r}, w={w} -> Weight must be even.")
                continue
            if w >= r:
                print(f"[Skipping] r={r}, w={w} -> Weight w must be < Block Size r.")
                continue
            
            for t in t_list:
                t = int(t)
                config_idx += 1
                
                print(f"\n[Set {config_idx}/{total_configs}] Parameters: r={r}, w={w}, t={t}")
                
                success_count = 0
                valid_trials = 0
                start_time_batch = time.time()
                
                for i in range(trials):
                    print(f"    Trial {i+1:>2}/{trials} ...", end=" ", flush=True)
                    
                    trial_start = time.time()
                    
                    # --- A. Generate System ---
                    # Removed try-except so we see CRASHES if they happen
                    H_pub, priv = bike_keygen(r, w)
                        
                    valid_trials += 1
                    
                    # --- B. Encapsulate ---
                    c, _ = bike_encapsulate(H_pub, t)
                    
                    # --- C. Attack ---
                    is_broken = bike_isd_check(H_pub, c, t, max_iters=max_iters)
                    
                    duration = time.time() - trial_start
                    
                    if is_broken:
                        success_count += 1
                        print(f"BROKEN  (in {duration:.2f}s)")
                    else:
                        print(f"SECURE  (in {duration:.2f}s)")
                
                # Log Summary
                if valid_trials > 0:
                    rate = float(success_count) / float(valid_trials)
                    batch_time = time.time() - start_time_batch
                    print(f"  >> Result: {rate:.0%} Success (Batch took {batch_time:.1f}s)")
                    
                    results_list.append({
                        'r': r,
                        'w': w,
                        't': t,
                        'success_rate': rate
                    })

    print("\n" + "="*65)
    print("Experiment Complete.")
    return pd.DataFrame(results_list)

print("[Testbed] function defined.")

[Testbed] function defined.


In [ ]:
# A. Define Ranges
# r_vals = [53, 59, 61, 67, 71, 73, 79, 83, 89]

r_vals = [59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 127, 137, 139, 149, 151, 157, 163, 167]
t_vals = [4, 6, 8]

# Weight w: Small even numbers (Toy keys are usually sparse)
# We test a few values to see if it impacts the attack
w_vals = [6] 

# B. Run
# Budget: 1000 iterations (Quick check)
df_results = run_bike_3param_experiment(r_vals, t_vals, w_vals, trials=10, max_iters=5000)

--- Starting BIKE Testbed (Robust Mode) ---
Total Configurations: 54
r values: [59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 127, 137, 139, 149, 151, 157, 163, 167]
w values: [6]
t values: [4, 6, 8]

[Set 1/54] Parameters: r=59, w=6, t=4
    Trial  1/10 ... BROKEN  (in 0.01s)
    Trial  2/10 ... BROKEN  (in 0.03s)
    Trial  3/10 ... BROKEN  (in 0.00s)
    Trial  4/10 ... BROKEN  (in 0.01s)
    Trial  5/10 ... BROKEN  (in 0.06s)
    Trial  6/10 ... BROKEN  (in 0.03s)
    Trial  7/10 ... BROKEN  (in 0.00s)
    Trial  8/10 ... BROKEN  (in 0.11s)
    Trial  9/10 ... BROKEN  (in 0.02s)
    Trial 10/10 ... BROKEN  (in 0.01s)
  >> Result: 100% Success (Batch took 0.3s)

[Set 2/54] Parameters: r=59, w=6, t=6
    Trial  1/10 ... BROKEN  (in 0.08s)
    Trial  2/10 ... BROKEN  (in 0.09s)
    Trial  3/10 ... BROKEN  (in 0.01s)
    Trial  4/10 ... BROKEN  (in 0.06s)
    Trial  5/10 ... BROKEN  (in 0.01s)
    Trial  6/10 ... BROKEN  (in 0.00s)
    Trial  7/10 ... SECURE  (in 0.18s)
    Trial  8/10 ... 

## Visualizations

In [ ]:
# ==========================================
# 4. VISUALIZATION
# ==========================================
if not df_results.empty:
    # Since we have 3 parameters, we should visualize r vs t
    # and average over w (or pick one w).
    # Theory says w shouldn't matter for *this* specific attack.
    
    # Let's filter for one weight (e.g., w=6) to get a clean 2D plot
    target_w = 6
    subset = df_results[df_results['w'] == target_w]
    
    if subset.empty and not df_results.empty:
        # If w=6 wasn't valid for some r, just take the first available w
        target_w = df_results['w'].iloc[0]
        subset = df_results[df_results['w'] == target_w]

    print(f"\nVisualizing results for Key Weight w={target_w}")
    
    pivot = subset.pivot_table(index='t', columns='r', values='success_rate')
    pivot = pivot.sort_index(ascending=False)
    
    plt.figure(figsize=(12, 8))
    sns.heatmap(
        pivot, 
        annot=True, 
        fmt=".0%", 
        cmap="RdYlGn", 
        cbar_kws={'label': 'ISD Success Rate'},
        linewidths=.5
    )
    plt.title(f'BIKE Attack Success (r vs t)\nFixed Key Weight w={target_w}')
    plt.xlabel('Block Size (r)')
    plt.ylabel('Error Weight (t)')
    plt.show()
    
else:
    print("No results found.")